In [10]:
import numpy as np
import random 
import math

In [ ]:
class Value:
    def __init__(self, data, _children=(), _op=""):
        self.data = data
        self.grad = 0 
        self._prev = set(_children)
        self._op = _op
        self._backward = lambda : None

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward

        return out
    
    def __sub__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data - other.data, (self, other), '-')
        return out
    
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.data += other.grad * out.grad
            other.data += self.data * out.grad
        out._backward = _backward
        return out
    
    def __pow__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data**other, (self, other), f"**{other}")

        def _backward():
            self.data += (other * self.data**(other-1)) * out.data
        out._backward = _backward
        

        return out
    


    def tanh(self):
        n = self.data
        t = (math.exp(2*n) - 1) / (math.exp(2*n) + 1)
        out = Value(t, (self, ), 'tanh')

        def _backward():            # 1 + tanh ** 2
            self.data += (1 - t**2) * out.data
        out._backward = _backward



    def backward(self):
        #topological order
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        self.grad = 1
        for v in reversed(topo):
            v._backward()




    
    def __repr__(self):
        return f"Value(data={self.data})"


In [4]:
h = 0.01
a = Value(2)
b = Value(-3)
c = Value(10)
d = a*b + c
print(d)

Value(data=4)


In [5]:
print(type(d))

<class '__main__.Value'>


In [6]:
d._prev

{Value(data=-6), Value(data=10)}

In [13]:
Value(a)

Value(data=8)